# MAITE Compatibility demo

This notebook contains an end-to-end demostration of Dioptra that can be run on any modern laptop.

## Setup

Below we import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected.

In [ ]:
# Import packages from the Python standard library
import importlib.util
import os
import sys
import pprint
import time
import warnings
from pathlib import Path


def register_python_source_file(module_name: str, filepath: Path) -> None:
    """Import a source file directly.

    Args:
        module_name: The module name to associate with the imported source file.
        filepath: The path to the source file.

    Notes:
        Adapted from the following implementation in the Python documentation:
        https://docs.python.org/3/library/importlib.html#importing-a-source-file-directly
    """
    spec = importlib.util.spec_from_file_location(module_name, str(filepath))
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)


# Filter out warning messages
warnings.filterwarnings("ignore")

# Experiment name
EXPERIMENT_NAME = "pytorch_maite"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:30080"

# Set DIOPTRA_RESTAPI_URI variable if not defined, used to connect to RESTful API service
os.environ["DIOPTRA_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Register the examples/scripts directory as a Python module
register_python_source_file("scripts", Path("..", "scripts", "__init__.py"))

from scripts.client import DioptraClient
from scripts.utils import make_tar

# Import third-party Python packages
import numpy as np
from mlflow.tracking import MlflowClient

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `src/MLproject` file.
To run these entrypoints within Dioptra's architecture, we need to package those files up into an archive and submit it to the Dioptra RESTful API to create a new job.
For convenience, we provide the `make_tar` helper function defined in `examples/scripts/utils.py`.

In [ ]:
def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

In [ ]:
make_tar(["src"], WORKFLOWS_TAR_GZ)

To connect with the endpoint, we will use a client class defined in the `examples/scripts/client.py` file that is able to connect with the Dioptra RESTful API using the HTTP protocol.
We connect using the client below.
The client uses the environment variable `DIOPTRA_RESTAPI_URI`, which we configured at the top of the notebook, to figure out how to connect to the Dioptra RESTful API.

The MlflowClient object is used to retrieve our results.

In [ ]:
restapi_client = DioptraClient()
mlflow_client = MlflowClient()

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [ ]:
!python ../scripts/register_task_plugins.py --force --plugins-dir ../task-plugins --api-url http://localhost:30080

In [ ]:
!python ../scripts/register_queues.py --api-url http://localhost:30080

In [ ]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

The `full_workflow` entry point tests basic MAITE functionality: load a dataset from huggingface, load a model from huggingface, load a metric from torchvision and run that metric on that model/dataset. It also saves the model into MLFlow.

In [ ]:
def get_output(res):
    while mlflow_run_id_is_not_known(res) or res['status'] != "finished":
        time.sleep(1)
        res = restapi_client.get_job_by_id(res["jobId"])
    out = mlflow_client.get_run(res["mlflowRunId"])
    pprint.pprint(out.data.metrics)

In [ ]:
response_test_metrics = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="full_workflow",
    entry_point_kwargs=" ".join([
        "-P dataset_name=cifar10",
        "-P subset=32"
    ]),
    queue="pytorch_cpu",
    timeout="1h",
)
pprint.pprint(response_test_metrics)

The `save_model` entry point loads a model from huggingface and saves it to MLFlow.

In [ ]:
response_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="save_model",
    entry_point_kwargs=" ".join([
    ]),
    queue="pytorch_cpu",
    timeout="1h",
)
pprint.pprint(response_model)

The `test_model` entrypoint loads the previously saved model from MLFlow into a MAITE-readable format, and then uses maite to test metrics and a dataset on it.

Note: Currently this saves the dataset to /dioptra/data/tmp - make sure the docker container has permissions to write to this area or change the location this is saved to. 

In [ ]:
while mlflow_run_id_is_not_known(response_model):
    time.sleep(1)
    response_model = restapi_client.get_job_by_id(response_model["jobId"])

response_use_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="test_model",
    entry_point_kwargs=" ".join([
        "-P model_name=loaded_model",
        "-P model_version=1",
        "-P subset=500"
    ]),
    queue="pytorch_cpu",
    timeout="1h",
)

The `load_dataset` entrypoint loads a dataset from disk, puts it into maite format, then loads a model and metric using maite and runs it on that dataset.

In [ ]:
response_load_dataset = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="load_dataset",
    entry_point_kwargs=" ".join([
        "-P subset=400"
    ]),
    queue="pytorch_cpu",
    timeout="1h",
)

The `gen` entrypoint loads a dataset using MAITE, runs the specified attack on it, saves the output of the attack to MLFlow as an artifact, and has the option to write the original dataset to disk (configurable). The `do_gen_attack` function takes the class name of the attack, as well as a dictionary of parameters. Any unnecessary parameters will be filtered out and reported in the logs.

Note: While this function in theory could work with poisoning attacks since it uses heart-lib, and does not have anything specifically requiring an evasion attack, at the time of writing there did not seem to exist a compatible poisoning example. This is largely due to poisoning examples requiring either knowledge of the feature layers from the model (and this information is not available in the context of this notebook) or in the case of PoisoningAttackBackdoor, due to an incompatibility with heartlib. It is possible that in the future this may be corrected and that example may work. 

In [ ]:
import json, shlex
def format_kwargs(library, name, kwargs_dict):
    jsd = json.dumps(kwargs_dict, separators=(',',':'))
    args = [
        "subset=100",
        "save_original=False",
        "batch_size=2",
        f"attack_name={name}",
        f"attack_library={library}",
        f"attack_kwargs={jsd}"
    ]
    
    cmdline = " ".join(
        "-P " + shlex.quote(arg) for arg in args
    )
    return cmdline
    
def do_gen_attack(attack_name, kwargs_dict={},  attack_library='art.attacks.evasion'):
    cmdline = format_kwargs(attack_library, attack_name, kwargs_dict)
    return restapi_client.submit_job(
        workflows_file=WORKFLOWS_TAR_GZ,
        experiment_name=EXPERIMENT_NAME,
        entry_point="gen",
        entry_point_kwargs=cmdline,
        queue="pytorch_cpu",
        timeout="1h",
    )


In [ ]:
response_gen_fgm = do_gen_attack("FastGradientMethod", {'eps': 0.3, 'eps_step': 0.1, 'norm': 'inf', 'minimal': False})

In [ ]:
# may require GPU
response_gen_pt = do_gen_attack("PixelAttack", {})

In [ ]:
# may require GPU
response_gen_pgd = do_gen_attack("ProjectedGradientDescentPyTorch", {})

In [ ]:
# may require GPU
response_gen_hsj = do_gen_attack("HopSkipJump", {})

In [ ]:
# does not work currently due to problem with heartlib, but includes syntax for passing an existing function as an argument, and also for using a different library
# response_gen_poison = do_gen_attack("PoisoningAttackBackdoor", {'perturbation_FUNCTION': 'art.attacks.poisoning.perturbations.add_single_bd' }, attack_library='art.attacks.poisoning')

The `infer` entrypoint takes the previously generated fgm attack results and runs it against a given model and metric. It is included here as a function and tested against 4 models on huggingface from different authors. Note that not all CIFAR10 targeted models on huggingface are compatible for various reasons - missing `config.json`, different requirements for data formatting, etc. The examples included below worked at the time of testing.

Although MAITE supports torchvision as a provider as well, torchvision does not seem to provide pretrained CIFAR10 models. An ImageNET example may be more suited to cross-testing torchvision and huggingface models.

In [ ]:
def test_cifar10_fgm(provider, model):
    global response_gen_fgm
    while mlflow_run_id_is_not_known(response_gen_fgm):
        time.sleep(1)
        response_gen_fgm = restapi_client.get_job_by_id(response_gen_fgm["jobId"])
    response_infer_fgm = restapi_client.submit_job(
        workflows_file=WORKFLOWS_TAR_GZ,
        experiment_name=EXPERIMENT_NAME,
        entry_point="infer",
        entry_point_kwargs=" ".join([
            f"-P run_id={response_gen_fgm['mlflowRunId']}",
            f"-P model_provider_name={provider}",
            f"-P model_name={model}",
            f"-P model_task=image-classification"
        ]),
        queue="pytorch_cpu",
        timeout="1h",
        depends_on=response_gen_fgm["jobId"],
    )
    return response_infer_fgm

In [ ]:
model1_results = test_cifar10_fgm("huggingface","aaraki/vit-base-patch16-224-in21k-finetuned-cifar10")
get_output(model1_results)

In [ ]:
model2_results = test_cifar10_fgm("huggingface","abhishek/autotrain_cifar10_vit_base")
get_output(model2_results)

In [ ]:
model3_results = test_cifar10_fgm("huggingface","Weili/vit-base-patch16-224-finetuned-cifar10")
get_output(model3_results)

In [ ]:
model4_results = test_cifar10_fgm("huggingface","arize-ai/resnet-50-cifar10-quality-drift")
get_output(model4_results)